In [1]:
from pymongo import MongoClient
import time
from datetime import datetime, timedelta

client = MongoClient('localhost', 27017)
db = client["test_mongo"]

# Actualizaciones a la base de datos con 1 millón de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 1 millón de documentos ya indexados

In [25]:
coll = db["extracto_cuenta_1m2"]

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [4]:
def actualizacion_fecha_monto(start_date, end_date, monto, coll):
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    
    criteria = {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    
    new = {
        "$set":
            {
                "monto": monto, 
                "nota": "Monto actualizado por fecha"
            }
        }

    return coll.update_many(criteria, new)

In [5]:
start_date = "2021-08-01"
end_date = "2021-08-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1152 documentos
 Actualización realizada en un tiempo de 0:00:45.321720


In [6]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db340dfb1bbbb1b844d21'), 'fecha': '2021-08-02 01:25:37', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 8837884.56, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db340dfb1bbbb1b8365b0'), 'fecha': '2021-08-02 02:08:28', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Deposito a cuenta', 'saldo': 535334.66, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db343dfb1bbbb1b8c7fba'), 'fecha': '2021-08-02 02:09:28', 'agencia': 'La Paz', 'monto': 7890, 'descripcion': 'Pago recibo de luz', 'saldo': 6480861.49, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db33fdfb1bbbb1b80a223'), 'fecha': '2021-08-02 02:40:03', 'agencia': 'Pando', 'monto': 7890, 'descripcion': 'Pago recibo de gas', 'saldo': 3829466.48, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db341dfb1bbbb1b866eef'), 'fecha': '2021-08-02 03:13:41', 'agencia': 'Beni', 'monto': 7890, 'descripcion': 'Pago en interés', 

## Actualización 2: Nota por descripción y fecha

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [26]:
def actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll):
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    
    criteria = {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
        }
    
    new = {
        "$set":
            {
                "monto": monto, 
                "nota": "Monto actualizado por descripción"
            }
        }

    return coll.update_many(criteria, new)

In [27]:
start_date = "1980-05-14"
end_date = "1980-05-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 27 documentos
 Actualización realizada en un tiempo de 0:00:05.853436


In [29]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db33edfb1bbbb1b7f063b'), 'fecha': '1980-05-15 08:32:48', 'agencia': 'Oruro', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1253093.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db33fdfb1bbbb1b80e07d'), 'fecha': '1980-05-16 17:50:02', 'agencia': 'La Paz', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 6428046.41, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db342dfb1bbbb1b8bdbc8'), 'fecha': '1980-05-16 23:32:03', 'agencia': 'Pando', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 2259769.54, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db341dfb1bbbb1b87cc9b'), 'fecha': '1980-05-18 03:14:46', 'agencia': 'Oruro', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 729783.05, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db343dfb1bbbb1b8c6b11'), 'fecha': '1980-05-18 21:55:15', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descr

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [30]:
def actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll):
    
    criteria = {
        "$and":
            [
                {
                    "monto": {"$lt": monto}
                },
                {
                    "descripcion": descripcion
                },
                {
                    "$or":
                        [
                            {
                                "agencia": agencia1
                            },
                            {
                                "agencia": agencia2
                            }  
                        ]
                }
            ]
        }
    
    new = {
        "$set":
            {
                "saldo": saldo, 
                "nota": "Saldo actualizado por monto y agencias"
            }
        }

    return coll.update_many(criteria, new)

In [38]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 34 documentos
 Actualización realizada en un tiempo de 0:00:01.197947


In [39]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db33edfb1bbbb1b7d6de2'), 'fecha': '1973-12-30 15:18:51', 'agencia': 'La Paz', 'monto': 1090.72, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b7fa0f8'), 'fecha': '1992-09-23 08:51:16', 'agencia': 'Santa Cruz', 'monto': 1183.89, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b7faa61'), 'fecha': '2019-05-05 09:15:43', 'agencia': 'Santa Cruz', 'monto': 1100.83, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b80004d'), 'fecha': '1964-02-17 19:22:34', 'agencia': 'La Paz', 'monto': 1803.68, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33fdfb1bbbb1b807973'), 'fecha': '1997-01-05 16:54:23', 'agencia': 'Santa Cruz', 'monto': 1687.35, 'descripcion': 'Interes

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 45.3217 s      |
| Tiempo actualización 2      | 5.8534 s       |
| Tiempo actualización 3      | 1.1979 s       |

# Actualizaciones a la base de datos con 10 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos ya indexados

In [40]:
coll = db["extracto_cuenta_10m2"]

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [42]:
start_date = "2021-12-01"
end_date = "2021-12-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 11439 documentos
 Actualización realizada en un tiempo de 0:04:33.459410


In [43]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db08fdfb1bba8ec4915df'), 'fecha': '2021-12-02 00:00:54', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'Pago de cheque compensado', 'saldo': 7895316.3, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db05edfb1bba8ec2b16dd'), 'fecha': '2021-12-02 00:01:56', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Pago recibo de agua', 'saldo': 1212965.69, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db0a9dfb1bba8ec5a1379'), 'fecha': '2021-12-02 00:04:40', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Deposito a cuenta', 'saldo': 1100613.5, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db013dfb1bba8ecfac0ce'), 'fecha': '2021-12-02 00:05:00', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Pago recibo de gas', 'saldo': 5367940.51, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db0abdfb1bba8ec5fa176'), 'fecha': '2021-12-02 00:09:02', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'DEB.CTA.P/C.INT

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [46]:
start_date = "1980-01-14"
end_date = "1980-04-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1241 documentos
 Actualización realizada en un tiempo de 0:04:00.600043


In [47]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db0a9dfb1bba8ec58cd80'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db017dfb1bba8ec01cc35'), 'fecha': '1980-01-15 02:24:13', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 1356620.84, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db092dfb1bba8ec4ecb6e'), 'fecha': '1980-01-15 02:24:49', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 215898.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db0abdfb1bba8ec600b09'), 'fecha': '1980-01-15 03:49:51', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1483142.98, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636dafe6dfb1bba8ecdcb513'), 'fecha': '1980-01-15 04:25:00', 'agencia': 'Cochabamba', 'monto': 10.5, 'de

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [48]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 423 documentos
 Actualización realizada en un tiempo de 0:02:20.916705


In [49]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db0a9dfb1bba8ec5833e4'), 'fecha': '1960-07-09 07:48:09', 'agencia': 'Santa Cruz', 'monto': 12.03, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636dafe9dfb1bba8ece2d0b0'), 'fecha': '2018-04-01 20:25:14', 'agencia': 'La Paz', 'monto': 16.2, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db0abdfb1bba8ec5f5b09'), 'fecha': '1964-04-12 18:53:53', 'agencia': 'Santa Cruz', 'monto': 19.62, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db02edfb1bba8ec0b2d76'), 'fecha': '1956-04-14 01:32:42', 'agencia': 'La Paz', 'monto': 20.34, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636dafe5dfb1bba8ecda6e7e'), 'fecha': '1964-09-10 17:22:52', 'agencia': 'Santa Cruz', 'monto': 21.98, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 45.3217 s      |
| Tiempo actualización 2      | 5.8534 s       |
| Tiempo actualización 3      | 1.1979 s       |

# Actualizaciones a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos ya indexados

In [58]:
coll = db["extracto_cuenta_30m"]

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [59]:
start_date = "2020-12-01"
end_date = "2020-12-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 34078 documentos
 Actualización realizada en un tiempo de 0:41:28.474775


In [60]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d40b3dfb1bb4dd6d924d5'), 'fecha': '2020-12-02 00:00:14', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'Pago de nómina', 'saldo': 458926.79, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d3f87dfb1bb4dd6074ee9'), 'fecha': '2020-12-02 00:01:05', 'agencia': 'Chuquisaca', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 5416583.29, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d40b1dfb1bb4dd6d319fa'), 'fecha': '2020-12-02 00:01:15', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 1716273.52, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d4126dfb1bb4dd620a662'), 'fecha': '2020-12-02 00:02:58', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Reintegro cajero automático', 'saldo': 1545910.97, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d41c3dfb1bb4dd67be379'), 'fecha': '2020-12-02 00:05:49', 'agencia': 'Cochabamba', 'monto': 7890, 'descri

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [61]:
start_date = "1980-01-14"
end_date = "1980-04-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 7737 documentos
 Actualización realizada en un tiempo de 0:28:45.103925


In [62]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d416edfb1bb4dd652c645'), 'fecha': '1980-01-15 00:03:48', 'agencia': 'Santa Cruz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1191798.41, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d413fdfb1bb4dd636de97'), 'fecha': '1980-01-15 00:16:58', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 4173801.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d3fe5dfb1bb4dd64aeb78'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d402bdfb1bb4dd6796f20'), 'fecha': '1980-01-15 01:06:53', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 8847975.29, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d40c9dfb1bb4dd6e575ef'), 'fecha': '1980-01-15 01:16:24', 'agencia': 'Cochabamba', 'monto': 10.5, 'descrip

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [64]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 100

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 68 documentos
 Actualización realizada en un tiempo de 0:00:31.925104


In [65]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d3fe5dfb1bb4dd64a51dc'), 'fecha': '1960-07-09 07:48:09', 'agencia': 'Santa Cruz', 'monto': 12.03, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3f31dfb1bb4dd6d4eea8'), 'fecha': '2018-04-01 20:25:14', 'agencia': 'La Paz', 'monto': 16.2, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d40f7dfb1bb4dd6fe4cef'), 'fecha': '1977-10-03 06:06:13', 'agencia': 'Santa Cruz', 'monto': 18.13, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3fe7dfb1bb4dd6517901'), 'fecha': '1964-04-12 18:53:53', 'agencia': 'Santa Cruz', 'monto': 19.62, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3f72dfb1bb4dd6fd4b6d'), 'fecha': '1956-04-14 01:32:42', 'agencia': 'La Paz', 'monto': 20.34, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido   |
|-----------------------------|------------------|
| Tiempo actualización 1      | 41 min 28.4748 s |
| Tiempo actualización 2      | 28 min 45.1039 s |
| Tiempo actualización 3      | 31.9251 s        |

# Actualizaciones a la base de datos con 10 millones de registros y solo tres campos indexados (fecha, descripción, nota)

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos ya indexados en tres campos: fecha, descripción, nota.

In [66]:
coll = db["extracto_cuenta_10m_3idx"]

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [67]:
start_date = "2020-12-01"
end_date = "2020-12-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 11413 documentos
 Actualización realizada en un tiempo de 0:08:55.191605


In [68]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e8328dfb1bb5ea6f29969'), 'fecha': '2020-12-02 00:01:05', 'agencia': 'Chuquisaca', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 5416583.29, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636e838adfb1bb5ea6328ca9'), 'fecha': '2020-12-02 00:14:50', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 7071478.84, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636e8344dfb1bb5ea60bf971'), 'fecha': '2020-12-02 00:16:15', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 1849627.55, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636e8342dfb1bb5ea60759a7'), 'fecha': '2020-12-02 00:16:34', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Aporte inversor', 'saldo': 5952786.9, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636e8359dfb1bb5ea61960d6'), 'fecha': '2020-12-02 00:30:11', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion':

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [69]:
start_date = "1980-01-14"
end_date = "1980-01-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 271 documentos
 Actualización realizada en un tiempo de 0:00:43.080891


In [70]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e838bdfb1bb5ea63635f9'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e82ffdfb1bb5ea6df34ad'), 'fecha': '1980-01-15 02:24:13', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 1356620.84, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e8377dfb1bb5ea62c33e7'), 'fecha': '1980-01-15 02:24:49', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 215898.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e838edfb1bb5ea63d7382'), 'fecha': '1980-01-15 03:49:51', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1483142.98, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e82d0dfb1bb5ea6ba1d8c'), 'fecha': '1980-01-15 04:25:00', 'agencia': 'Cochabamba', 'monto': 10.5, 'de

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [71]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 200

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 48 documentos
 Actualización realizada en un tiempo de 0:01:24.285184


In [72]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e82b9dfb1bb5ea6aa7299'), 'fecha': '1955-01-30 00:27:19', 'agencia': 'Santa Cruz', 'monto': 155.16, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e82badfb1bb5ea6ac06ba'), 'fecha': '2019-01-07 04:12:30', 'agencia': 'Santa Cruz', 'monto': 115.47, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e82bbdfb1bb5ea6af6c80'), 'fecha': '2011-07-25 02:59:35', 'agencia': 'La Paz', 'monto': 154.42, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e82bbdfb1bb5ea6afcdfc'), 'fecha': '1989-05-31 12:37:13', 'agencia': 'La Paz', 'monto': 60.51, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e82bcdfb1bb5ea6b29fc7'), 'fecha': '1977-11-10 05:02:53', 'agencia': 'La Paz', 'monto': 74.07, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido  |
|-----------------------------|-----------------|
| Tiempo actualización 1      | 8 min 55.1916 s |
| Tiempo actualización 2      | 43.0809 s       |
| Tiempo actualización 3      | 1 min 24.2851 s |

# Actualizaciones a la base de datos con 30 millones de registros y solo tres campos indexados (fecha, descripción, nota)

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos ya indexados en tres campos: fecha, descripción, nota.

In [73]:
coll = db["extracto_cuenta_30m_3idx"]

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [74]:
start_date = "2020-12-01"
end_date = "2020-12-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 34078 documentos
 Actualización realizada en un tiempo de 0:32:01.650369


In [77]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e7b48dfb1bb4100ea2597'), 'fecha': '1980-01-15 00:03:48', 'agencia': 'Santa Cruz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1191798.41, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e7a6adfb1bb41005e238b'), 'fecha': '1980-01-15 00:06:57', 'agencia': 'Tarija', 'monto': 103475.52, 'descripcion': 'Pago recibo de luz', 'saldo': 5772881.68, 'nota': 'dignissimos ab a corrupti repellendus repudiandae optio ipsa'}
{'_id': ObjectId('636e78f0dfb1bb410075f3a3'), 'fecha': '1980-01-15 00:08:33', 'agencia': 'Chuquisaca', 'monto': 32501.86, 'descripcion': 'Debito Cta por compra', 'saldo': 1567659.37, 'nota': 'fugit id reprehenderit saepe magnam neque et in cumque aspernatur'}
{'_id': ObjectId('636e7923dfb1bb41009ad570'), 'fecha': '1980-01-15 00:10:09', 'agencia': 'Santa Cruz', 'monto': 264585.37, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 416943.96, 'nota': 'inventore deserunt rerum iste ducimus ratione animi soluta esse adipisci 

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [75]:
start_date = "1980-01-14"
end_date = "1980-01-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 807 documentos
 Actualización realizada en un tiempo de 0:02:56.443795


In [76]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e7b48dfb1bb4100ea2597'), 'fecha': '1980-01-15 00:03:48', 'agencia': 'Santa Cruz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1191798.41, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e7b1ddfb1bb4100ce3de9'), 'fecha': '1980-01-15 00:16:58', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 4173801.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e799edfb1bb4100e24aca'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e79e8dfb1bb410010ce73'), 'fecha': '1980-01-15 01:06:53', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 8847975.29, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636e7a9edfb1bb41007cd542'), 'fecha': '1980-01-15 01:16:24', 'agencia': 'Cochabamba', 'monto': 10.5, 'descrip

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [78]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 200

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion, coll)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 136 documentos
 Actualización realizada en un tiempo de 0:07:47.596264


In [80]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636e78c0dfb1bb410056876b'), 'fecha': '1955-01-30 00:27:19', 'agencia': 'Santa Cruz', 'monto': 155.16, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e78c0dfb1bb4100581b8c'), 'fecha': '2019-01-07 04:12:30', 'agencia': 'Santa Cruz', 'monto': 115.47, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e78c2dfb1bb41005b8152'), 'fecha': '2011-07-25 02:59:35', 'agencia': 'La Paz', 'monto': 154.42, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e78c2dfb1bb41005be2ce'), 'fecha': '1989-05-31 12:37:13', 'agencia': 'La Paz', 'monto': 60.51, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636e78c3dfb1bb41005eb499'), 'fecha': '1977-11-10 05:02:53', 'agencia': 'La Paz', 'monto': 74.07, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido   |
|-----------------------------|------------------|
| Tiempo actualización 1      | 32 min 01.6503 s |
| Tiempo actualización 2      | 2 min 56.4438 s  |
| Tiempo actualización 3      | 7 min 47.5962 s  |